In [1]:
import pandas as pd
import os

In [2]:
# Đường dẫn thư mục chứa các file Excel
base_path = 'Data/Data raw'

In [3]:
# Đọc từng file Excel và gán vào các biến tương ứng
data1 = pd.read_excel(f'{base_path}/2020-Vietnam.xlsx')
data2 = pd.read_excel(f'{base_path}/2021-Vietnam.xlsx')
data3 = pd.read_excel(f'{base_path}/2022-Vietnam.xlsx')
data4 = pd.read_excel(f'{base_path}/2023-Vietnam.xlsx')
data5 = pd.read_excel(f'{base_path}/2024-Vietnam.xlsx')

In [4]:
# Kiểm tra danh sách các cột
print(data1.columns)

Index(['Mã', 'Tên công ty', 'Sàn', 'Ngành ICB - cấp 1', 'Ngành ICB - cấp 2',
       'Ngành ICB - cấp 3', 'Ngành ICB - cấp 4',
       'Quý\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\n',
       'Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\n',
       'Trạng thái kiểm toán\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\n',
       ...
       'TM. Chi phí tài chính khác\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\nĐơn vị: Triệu VND',
       'TM. Chi phí sản xuất theo yếu tố\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\nĐơn vị: Triệu VND',
       'TM. Chi phí nguyên liệu, vật liệu\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\nĐơn vị: Triệu VND',
       'TM. Chi phí nhân công\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\nĐơn vị: Triệu VND',
       'TM.Chi phí khấu hao tài sản cố định\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\nĐơn vị: Triệu VND',
       'TM. Chi phí dịch vụ mua ngoài\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\nĐơn vị: Triệu VND',
       'TM. Chi phí khác bằng tiền\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\nĐơn vị: Triệu VND',
       'BCTCKH. Doanh thu 

In [5]:
# Lọc các cột có chữ "TM." ở đầu
columns_to_select = ["Mã"] + [col for col in data1.columns if col.startswith("TM.")]

# Nếu cột "Năm" tồn tại, thêm vào danh sách
if "Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\n" in data1.columns:
    columns_to_select.insert(1, "Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\n")  # Thêm "Năm" vào vị trí thứ hai

# Nếu biến year_column đã được định nghĩa và tồn tại trong data1, thêm vào danh sách
if 'year_column' in globals() and year_column in data1.columns:
    columns_to_select.insert(1, year_column)

# Tạo DataFrame mới chỉ chứa các cột cần thiết
data1_TM = data1[columns_to_select]

# Loại bỏ các ký tự không mong muốn khỏi tên cột
data1_TM.columns = data1_TM.columns.str.replace("\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\n", "", regex=False)
data1_TM.columns = data1_TM.columns.str.replace("Đơn vị: Triệu VND", "", regex=False)
data1_TM.columns = data1_TM.columns.str.replace("TM.", "", regex=False)

# Loại bỏ cột "Quý" nếu tồn tại
if "Quý" in data1_TM.columns:
    data1_TM = data1_TM.drop(columns=["Quý"])

# Đổi tên cột chứa thông tin năm thành "Năm"
if 'year_column' in globals() and year_column in data1.columns:
    data1_TM = data1_TM.rename(columns={year_column: "Năm"})

# Thay thế các giá trị NaN hoặc trống bằng 0
data1_TM = data1_TM.fillna(0)

# Chuyển giá trị trong cột "Năm" thành số nguyên (nếu cột "Năm" tồn tại)
if "Năm" in data1_TM.columns:
    data1_TM["Năm"] = data1_TM["Năm"].astype(str).str.extract(r'(\d+)').astype(int)

# Hiển thị DataFrame sau khi xử lý
data1_TM.head()


,Mã,Năm,Tiền,Tiền mặt,Tiền gửi Ngân hàng,Tiền đang chuyển,Tiền và tương đương tiền,Đầu tư tài chính NH,Chứng khoán đầu tư ngắn hạn,Đầu tư nắm giữ đến ngày đáo hạn,...,Lỗ chênh lệch tỷ giá đã thực hiện,Lỗ chênh lệch tỷ giá chưa thực hiện,"Dự phòng giảm giá các khoản đầu tư ngắn hạn, dài hạn",Chi phí tài chính khác,Chi phí sản xuất theo yếu tố,"Chi phí nguyên liệu, vật liệu",Chi phí nhân công,Chi phí khấu hao tài sản cố định,Chi phí dịch vụ mua ngoài,Chi phí khác bằng tiền
0,ACB,2020,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,BCM,2020,1.775587e+12,1.047283e+10,3.343554e+11,58277272.0,1.430700e+12,6.449500e+10,0.0,6.449500e+10,...,4763535.0,388272.0,0.0,1.720205e+10,6.514457e+12,1.199686e+12,9.695249e+11,2.380000e+11,3.475601e+12,6.316454e+11
2,BID,2020,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,BVH,2020,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,CTG,2020,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [7]:
# Lọc các cột có chữ "TM." ở đầu
columns_to_select = ["Mã"] + [col for col in data2.columns if col.startswith("TM.")]

# Nếu cột "Năm" tồn tại, thêm vào danh sách
if "Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2021\n" in data2.columns:
    columns_to_select.insert(1, "Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2021\n")  # Thêm "Năm" vào vị trí thứ hai

# Tạo DataFrame mới chỉ chứa các cột cần thiết
data2_TM = data2[columns_to_select]

# Loại bỏ các ký tự không mong muốn khỏi tên cột
data2_TM.columns = data2_TM.columns.str.replace("\nHợp nhất\nQuý: Hàng năm\nNăm: 2021\n", "", regex=False)
data2_TM.columns = data2_TM.columns.str.replace("Đơn vị: Triệu VND", "", regex=False)
data2_TM.columns = data2_TM.columns.str.replace("TM.", "", regex=False)

# Loại bỏ cột "Quý" nếu tồn tại
if "Quý" in data2_TM.columns:
    data2_TM = data2_TM.drop(columns=["Quý"])

# Thay thế các giá trị NaN hoặc trống bằng 0
data2_TM = data2_TM.fillna(0)

# Chuyển giá trị trong cột "Năm" thành số nguyên
if "Năm" in data2_TM.columns:
    data2_TM["Năm"] = data2_TM["Năm"].astype(str).str.extract(r'(\d+)').astype(int)

# Hiển thị DataFrame sau khi xử lý
data2_TM.head()


,Mã,Năm,Tiền,Tiền mặt,Tiền gửi Ngân hàng,Tiền đang chuyển,Tiền và tương đương tiền,Đầu tư tài chính NH,Chứng khoán đầu tư ngắn hạn,Đầu tư nắm giữ đến ngày đáo hạn,...,Lỗ chênh lệch tỷ giá đã thực hiện,Lỗ chênh lệch tỷ giá chưa thực hiện,"Dự phòng giảm giá các khoản đầu tư ngắn hạn, dài hạn",Chi phí tài chính khác,Chi phí sản xuất theo yếu tố,"Chi phí nguyên liệu, vật liệu",Chi phí nhân công,Chi phí khấu hao tài sản cố định,Chi phí dịch vụ mua ngoài,Chi phí khác bằng tiền
0,ACB,2021,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BCM,2021,2.975350e+12,1.188501e+10,3.234375e+11,27110000.0,2.640000e+12,2.240900e+11,0.0,2.240900e+11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BID,2021,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BVH,2021,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CTG,2021,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Lọc các cột có chữ "TM." ở đầu
columns_to_select = ["Mã"] + [col for col in data3.columns if col.startswith("TM.")]

# Nếu cột "Năm" tồn tại, thêm vào danh sách
if "Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2022\n" in data3.columns:
    columns_to_select.insert(1, "Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2022\n")  # Thêm "Năm" vào vị trí thứ hai

# Tạo DataFrame mới chỉ chứa các cột cần thiết
data3_TM = data3[columns_to_select]

# Loại bỏ các ký tự không mong muốn khỏi tên cột
data3_TM.columns = data3_TM.columns.str.replace("\nHợp nhất\nQuý: Hàng năm\nNăm: 2022\n", "", regex=False)
data3_TM.columns = data3_TM.columns.str.replace("Đơn vị: Triệu VND", "", regex=False)
data3_TM.columns = data3_TM.columns.str.replace("TM.", "", regex=False)

# Loại bỏ cột "Quý" nếu tồn tại
if "Quý" in data3_TM.columns:
    data3_TM = data3_TM.drop(columns=["Quý"])

# Thay thế các giá trị NaN hoặc trống bằng 0
data3_TM = data3_TM.fillna(0)

# Chuyển giá trị trong cột "Năm" thành số nguyên (nếu cột "Năm" tồn tại)
if "Năm" in data3_TM.columns:
    data3_TM["Năm"] = data3_TM["Năm"].astype(str).str.extract(r'(\d+)').astype(int)

# Hiển thị DataFrame sau khi xử lý
data3_TM.head()


,Mã,Năm,Tiền,Tiền mặt,Tiền gửi Ngân hàng,Tiền đang chuyển,Tiền và tương đương tiền,Đầu tư tài chính NH,Chứng khoán đầu tư ngắn hạn,Đầu tư nắm giữ đến ngày đáo hạn,...,Lỗ chênh lệch tỷ giá đã thực hiện,Lỗ chênh lệch tỷ giá chưa thực hiện,"Dự phòng giảm giá các khoản đầu tư ngắn hạn, dài hạn",Chi phí tài chính khác,Chi phí sản xuất theo yếu tố,"Chi phí nguyên liệu, vật liệu",Chi phí nhân công,Chi phí khấu hao tài sản cố định,Chi phí dịch vụ mua ngoài,Chi phí khác bằng tiền
0,ACB,2022,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,BCM,2022,1.070647e+12,1.061949e+10,8.809138e+11,3500000.0,1.791099e+11,4.364951e+11,4.364951e+11,0.0,...,0.0,0.0,0.0,194442752.0,5.341905e+12,5.733107e+11,8.659910e+11,1.897133e+11,3.386323e+12,3.265673e+11
2,BID,2022,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,BVH,2022,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,CTG,2022,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [9]:
# Lọc các cột có chữ "TM." ở đầu
columns_to_select = ["Mã"] + [col for col in data4.columns if col.startswith("TM.")]

# Nếu cột "Năm" tồn tại, thêm vào danh sách
if "Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2023\n" in data4.columns:
    columns_to_select.insert(1, "Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2023\n")  # Thêm "Năm" vào vị trí thứ hai

# Tạo DataFrame mới chỉ chứa các cột cần thiết
data4_TM = data4[columns_to_select]

# Loại bỏ các ký tự không mong muốn khỏi tên cột
data4_TM.columns = data4_TM.columns.str.replace("\nHợp nhất\nQuý: Hàng năm\nNăm: 2023\n", "", regex=False)
data4_TM.columns = data4_TM.columns.str.replace("Đơn vị: Tỷ VND", "", regex=False)
data4_TM.columns = data4_TM.columns.str.replace("TM.", "", regex=False)

# Loại bỏ cột "Quý" nếu tồn tại
if "Quý" in data4_TM.columns:
    data4_TM = data4_TM.drop(columns=["Quý"])

# Thay thế các giá trị NaN hoặc trống bằng 0
data4_TM = data4_TM.fillna(0)

# Chuyển giá trị trong cột "Năm" thành số nguyên (nếu cột "Năm" tồn tại)
if "Năm" in data4_TM.columns:
    data4_TM["Năm"] = data4_TM["Năm"].astype(str).str.extract(r'(\d+)').astype(int)

# Lấy danh sách các cột cần chuyển đổi (ngoại trừ "Mã" và "Năm")
cols_to_convert = [col for col in data4_TM.columns if col not in ["Mã", "Năm"]]

# Nhân giá trị của các cột đó với 1000
data4_TM[cols_to_convert] = data4_TM[cols_to_convert].apply(lambda x: x * 1000)

# Hiển thị DataFrame sau khi xử lý
data4_TM.head()


,Mã,Năm,Tiền,Tiền mặt,Tiền gửi Ngân hàng,Tiền đang chuyển,Tiền và tương đương tiền,Đầu tư tài chính NH,Chứng khoán đầu tư ngắn hạn,Đầu tư nắm giữ đến ngày đáo hạn,...,Lỗ chênh lệch tỷ giá đã thực hiện,Lỗ chênh lệch tỷ giá chưa thực hiện,"Dự phòng giảm giá các khoản đầu tư ngắn hạn, dài hạn",Chi phí tài chính khác,Chi phí sản xuất theo yếu tố,"Chi phí nguyên liệu, vật liệu",Chi phí nhân công,Chi phí khấu hao tài sản cố định,Chi phí dịch vụ mua ngoài,Chi phí khác bằng tiền
0,ACB,2023,0.000000e+00,0.000000,0.000000e+00,0.000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000
1,BCM,2023,1.332254e+06,12976.277891,1.132554e+06,23.816,186700.0,87314.044988,87314.044988,0.0,...,0.0,0.0,0.0,236.784738,4.841691e+06,810274.782829,836132.962813,264018.118642,2.490225e+06,441040.455978
2,BID,2023,0.000000e+00,0.000000,0.000000e+00,0.000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000
3,BVH,2023,0.000000e+00,0.000000,0.000000e+00,0.000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000
4,CTG,2023,0.000000e+00,0.000000,0.000000e+00,0.000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000


In [10]:
# Lọc các cột có chữ "TM." ở đầu
columns_to_select = ["Mã"] + [col for col in data5.columns if col.startswith("TM.")]

# Nếu cột "Năm" tồn tại, thêm vào danh sách
if "Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2024\n" in data5.columns:
    columns_to_select.insert(1, "Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2024\n")  # Thêm "Năm" vào vị trí thứ hai

# Tạo DataFrame mới chỉ chứa các cột cần thiết
data5_TM = data5[columns_to_select]

# Loại bỏ các ký tự không mong muốn khỏi tên cột
data5_TM.columns = data5_TM.columns.str.replace("\nHợp nhất\nQuý: Hàng năm\nNăm: 2024\n", "", regex=False)
data5_TM.columns = data5_TM.columns.str.replace("Đơn vị: Tỷ VND", "", regex=False)
data5_TM.columns = data5_TM.columns.str.replace("TM.", "", regex=False)

# Loại bỏ cột "Quý" nếu tồn tại
if "Quý" in data5_TM.columns:
    data5_TM = data5_TM.drop(columns=["Quý"])

# Thay thế các giá trị NaN hoặc trống bằng 0
data5_TM = data5_TM.fillna(0)

# Chuyển giá trị trong cột "Năm" thành số nguyên (nếu cột "Năm" tồn tại)
if "Năm" in data5_TM.columns:
    data5_TM["Năm"] = data5_TM["Năm"].astype(str).str.extract(r'(\d+)').astype(int)

# Lấy danh sách các cột cần chuyển đổi (ngoại trừ "Mã" và "Năm")
cols_to_convert = [col for col in data5_TM.columns if col not in ["Mã", "Năm"]]

# Nhân giá trị của các cột đó với 1000
data5_TM[cols_to_convert] = data5_TM[cols_to_convert].apply(lambda x: x * 1000)

# Hiển thị DataFrame sau khi xử lý
data5_TM.head()


,Mã,Năm,Tiền,Tiền mặt,Tiền gửi Ngân hàng,Tiền đang chuyển,Tiền và tương đương tiền,Đầu tư tài chính NH,Chứng khoán đầu tư ngắn hạn,Đầu tư nắm giữ đến ngày đáo hạn,...,Lỗ chênh lệch tỷ giá đã thực hiện,Lỗ chênh lệch tỷ giá chưa thực hiện,"Dự phòng giảm giá các khoản đầu tư ngắn hạn, dài hạn",Chi phí tài chính khác,Chi phí sản xuất theo yếu tố,"Chi phí nguyên liệu, vật liệu",Chi phí nhân công,Chi phí khấu hao tài sản cố định,Chi phí dịch vụ mua ngoài,Chi phí khác bằng tiền
0,ACB,2024,0.000000e+00,0.000000,0.000000e+00,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000
1,BCM,2024,2.210164e+06,5604.488272,2.067434e+06,25.1,137100.0,294912.603095,294912.603095,0.0,...,0.0,0.0,0.0,15371.859295,5.771417e+06,639836.339532,1.006558e+06,250963.180603,3.285210e+06,588849.512083
2,BID,2024,0.000000e+00,0.000000,0.000000e+00,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000
3,BVH,2024,0.000000e+00,0.000000,0.000000e+00,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000
4,CTG,2024,0.000000e+00,0.000000,0.000000e+00,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000


In [11]:
# Đường dẫn thư mục xuất file
output_dir = r"Data\Data cleaned"
os.makedirs(output_dir, exist_ok=True)

# Tạo đường dẫn file Excel mới với tên "data_TM.xlsx"
output_file = os.path.join(output_dir, "data_TM.xlsx")

# Xuất các DataFrame data1_TM đến data5_TM vào file Excel với các sheet riêng biệt
with pd.ExcelWriter(output_file) as writer:
    data1_TM.to_excel(writer, sheet_name="data1_TM", index=False)
    data2_TM.to_excel(writer, sheet_name="data2_TM", index=False)
    data3_TM.to_excel(writer, sheet_name="data3_TM", index=False)
    data4_TM.to_excel(writer, sheet_name="data4_TM", index=False)
    data5_TM.to_excel(writer, sheet_name="data5_TM", index=False)

print(f"File Excel đã được lưu tại: {output_file}")


File Excel đã được lưu tại: Data\Data cleaned\data_TM.xlsx
